In [ ]:
import cv2
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt

# --- 1. Baca gambar ---
img1 = cv2.imread("hutan1.jpg")
img2 = cv2.imread("hutan2.jpg")

# Samakan ukuran biar fair
img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

# --- 2. Ubah ke grayscale ---
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# --- 3. Hitung SSIM ---
score, diff = ssim(gray1, gray2, full=True)
print(f"Skor SSIM (0-1): {score:.4f}")

# --- 4. Visualisasi hasil perbedaan ---
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
plt.title("Desain 1")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.title("Desain 2")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(diff, cmap='gray')
plt.title("Perbedaan (SSIM diff map)")
plt.axis("off")

plt.show()


In [ ]:
import cv2
import os
import pandas as pd
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt

# --- Setup paths ---
test_folder = "../test"
results = []

# --- Loop semua subfolder di test ---
folders = sorted([f for f in os.listdir(test_folder) if os.path.isdir(os.path.join(test_folder, f)) and not f.startswith('.')])

for folder_name in folders:
    folder_path = os.path.join(test_folder, folder_name)
    
    # Cari file referensi (.jpg)
    jpg_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    if not jpg_files:
        print(f"⚠️ Tidak ada file .jpg di {folder_name}, skip...")
        continue
    
    reference_img_path = os.path.join(folder_path, jpg_files[0])
    bootstrap_img_path = os.path.join(folder_path, "bootstrap.png")
    tailwind_img_path = os.path.join(folder_path, "tailwind.png")
    
    # Cek apakah file ada
    if not os.path.exists(bootstrap_img_path) or not os.path.exists(tailwind_img_path):
        print(f"⚠️ File bootstrap.png atau tailwind.png tidak ada di {folder_name}, skip...")
        continue
    
    # --- 1. Baca gambar referensi ---
    img_ref = cv2.imread(reference_img_path)
    if img_ref is None:
        print(f"⚠️ Gagal membaca {reference_img_path}, skip...")
        continue
    
    # --- 2. Komparasi dengan Bootstrap ---
    img_bootstrap = cv2.imread(bootstrap_img_path)
    if img_bootstrap is not None:
        # Samakan ukuran
        img_bootstrap_resized = cv2.resize(img_bootstrap, (img_ref.shape[1], img_ref.shape[0]))
        
        # Ubah ke grayscale
        gray_ref = cv2.cvtColor(img_ref, cv2.COLOR_BGR2GRAY)
        gray_bootstrap = cv2.cvtColor(img_bootstrap_resized, cv2.COLOR_BGR2GRAY)
        
        # Hitung SSIM
        score_bootstrap, diff_bootstrap = ssim(gray_ref, gray_bootstrap, full=True)
        print(f"✅ {folder_name} - Bootstrap SSIM: {score_bootstrap:.4f}")
    else:
        score_bootstrap = None
        print(f"⚠️ Gagal membaca bootstrap.png di {folder_name}")
    
    # --- 3. Komparasi dengan Tailwind ---
    img_tailwind = cv2.imread(tailwind_img_path)
    if img_tailwind is not None:
        # Samakan ukuran
        img_tailwind_resized = cv2.resize(img_tailwind, (img_ref.shape[1], img_ref.shape[0]))
        
        # Ubah ke grayscale
        gray_ref = cv2.cvtColor(img_ref, cv2.COLOR_BGR2GRAY)
        gray_tailwind = cv2.cvtColor(img_tailwind_resized, cv2.COLOR_BGR2GRAY)
        
        # Hitung SSIM
        score_tailwind, diff_tailwind = ssim(gray_ref, gray_tailwind, full=True)
        print(f"✅ {folder_name} - Tailwind SSIM: {score_tailwind:.4f}")
    else:
        score_tailwind = None
        print(f"⚠️ Gagal membaca tailwind.png di {folder_name}")
    
    # Simpan hasil
    results.append({
        'folder': folder_name,
        'kategori': folder_name.split('-')[0],  # rendah, sedang, tinggi
        'nomor': folder_name.split('-')[1],
        'ssim_bootstrap': score_bootstrap,
        'ssim_tailwind': score_tailwind
    })

# --- 4. Simpan hasil ke CSV ---
df = pd.DataFrame(results)
df.to_csv('ssim_results.csv', index=False)
print(f"\n📊 Hasil disimpan ke ssim_results.csv")
print(df)

# --- 5. Visualisasi ringkasan per kategori ---
plt.figure(figsize=(12, 6))

categories = ['rendah', 'sedang', 'tinggi']
bootstrap_means = []
tailwind_means = []

for cat in categories:
    cat_data = df[df['kategori'] == cat]
    bootstrap_means.append(cat_data['ssim_bootstrap'].mean())
    tailwind_means.append(cat_data['ssim_tailwind'].mean())

x = range(len(categories))
width = 0.35

plt.bar([i - width/2 for i in x], bootstrap_means, width, label='Bootstrap', alpha=0.8)
plt.bar([i + width/2 for i in x], tailwind_means, width, label='Tailwind', alpha=0.8)

plt.xlabel('Kategori Kompleksitas')
plt.ylabel('Rata-rata SSIM Score')
plt.title('Perbandingan SSIM: Bootstrap vs Tailwind')
plt.xticks(x, categories)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('ssim_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📈 Grafik perbandingan disimpan ke ssim_comparison.png")

In [ ]:
import cv2
import os
import pandas as pd
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt

# --- Setup paths ---
test_folder = "../test"
results = []

# --- Loop semua subfolder di test ---
folders = sorted([f for f in os.listdir(test_folder) if os.path.isdir(os.path.join(test_folder, f)) and not f.startswith('.')])

for folder_name in folders:
    folder_path = os.path.join(test_folder, folder_name)
    
    # Cari file referensi (.jpg)
    jpg_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    if not jpg_files:
        print(f"⚠️ Tidak ada file .jpg di {folder_name}, skip...")
        continue
    
    reference_img_path = os.path.join(folder_path, jpg_files[0])
    bootstrap_img_path = os.path.join(folder_path, "bootstrap.png")
    tailwind_img_path = os.path.join(folder_path, "tailwind.png")
    
    # Cek apakah file ada
    if not os.path.exists(bootstrap_img_path) or not os.path.exists(tailwind_img_path):
        print(f"⚠️ File bootstrap.png atau tailwind.png tidak ada di {folder_name}, skip...")
        continue
    
    # --- 1. Baca gambar referensi ---
    img_ref = cv2.imread(reference_img_path)
    if img_ref is None:
        print(f"⚠️ Gagal membaca {reference_img_path}, skip...")
        continue
    
    # --- 2. Komparasi dengan Bootstrap ---
    img_bootstrap = cv2.imread(bootstrap_img_path)
    if img_bootstrap is not None:
        # Samakan ukuran
        img_bootstrap_resized = cv2.resize(img_bootstrap, (img_ref.shape[1], img_ref.shape[0]))
        
        # Ubah ke grayscale
        gray_ref = cv2.cvtColor(img_ref, cv2.COLOR_BGR2GRAY)
        gray_bootstrap = cv2.cvtColor(img_bootstrap_resized, cv2.COLOR_BGR2GRAY)
        
        # Hitung SSIM
        score_bootstrap, diff_bootstrap = ssim(gray_ref, gray_bootstrap, full=True)
        print(f"✅ {folder_name} - Bootstrap SSIM: {score_bootstrap:.4f}")
        
        # Visualisasi Bootstrap
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 4, 1)
        plt.imshow(cv2.cvtColor(img_ref, cv2.COLOR_BGR2RGB))
        plt.title("Referensi (Desain Acuan)")
        plt.axis("off")
        
        plt.subplot(1, 4, 2)
        plt.imshow(cv2.cvtColor(img_bootstrap_resized, cv2.COLOR_BGR2RGB))
        plt.title("Bootstrap")
        plt.axis("off")
        
        plt.subplot(1, 4, 3)
        plt.imshow(diff_bootstrap, cmap='gray')
        plt.title("Diff Map")
        plt.axis("off")
        
        plt.subplot(1, 4, 4)
        plt.text(0.5, 0.5, f'SSIM Score:\n{score_bootstrap:.4f}', 
                horizontalalignment='center', verticalalignment='center',
                fontsize=20, fontweight='bold', transform=plt.gca().transAxes)
        plt.axis("off")
        
        plt.suptitle(f'{folder_name} - Bootstrap Comparison', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    else:
        score_bootstrap = None
        diff_bootstrap = None
        print(f"⚠️ Gagal membaca bootstrap.png di {folder_name}")
    
    # --- 3. Komparasi dengan Tailwind ---
    img_tailwind = cv2.imread(tailwind_img_path)
    if img_tailwind is not None:
        # Samakan ukuran
        img_tailwind_resized = cv2.resize(img_tailwind, (img_ref.shape[1], img_ref.shape[0]))
        
        # Ubah ke grayscale
        gray_ref = cv2.cvtColor(img_ref, cv2.COLOR_BGR2GRAY)
        gray_tailwind = cv2.cvtColor(img_tailwind_resized, cv2.COLOR_BGR2GRAY)
        
        # Hitung SSIM
        score_tailwind, diff_tailwind = ssim(gray_ref, gray_tailwind, full=True)
        print(f"✅ {folder_name} - Tailwind SSIM: {score_tailwind:.4f}")
        
        # Visualisasi Tailwind
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 4, 1)
        plt.imshow(cv2.cvtColor(img_ref, cv2.COLOR_BGR2RGB))
        plt.title("Referensi (Desain Acuan)")
        plt.axis("off")
        
        plt.subplot(1, 4, 2)
        plt.imshow(cv2.cvtColor(img_tailwind_resized, cv2.COLOR_BGR2RGB))
        plt.title("Tailwind")
        plt.axis("off")
        
        plt.subplot(1, 4, 3)
        plt.imshow(diff_tailwind, cmap='gray')
        plt.title("Diff Map")
        plt.axis("off")
        
        plt.subplot(1, 4, 4)
        plt.text(0.5, 0.5, f'SSIM Score:\n{score_tailwind:.4f}', 
                horizontalalignment='center', verticalalignment='center',
                fontsize=20, fontweight='bold', transform=plt.gca().transAxes)
        plt.axis("off")
        
        plt.suptitle(f'{folder_name} - Tailwind Comparison', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    else:
        score_tailwind = None
        diff_tailwind = None
        print(f"⚠️ Gagal membaca tailwind.png di {folder_name}")
    
    # Simpan hasil
    results.append({
        'folder': folder_name,
        'kategori': folder_name.split('-')[0],  # rendah, sedang, tinggi
        'nomor': folder_name.split('-')[1],
        'ssim_bootstrap': score_bootstrap,
        'ssim_tailwind': score_tailwind
    })
    
    print("-" * 80)

# --- 4. Simpan hasil ke CSV ---
df = pd.DataFrame(results)
df.to_csv('ssim_results.csv', index=False)
print(f"\n📊 Hasil disimpan ke ssim_results.csv")
print(df)

# --- 5. Visualisasi ringkasan per kategori ---
plt.figure(figsize=(12, 6))

categories = ['rendah', 'sedang', 'tinggi']
bootstrap_means = []
tailwind_means = []

for cat in categories:
    cat_data = df[df['kategori'] == cat]
    bootstrap_means.append(cat_data['ssim_bootstrap'].mean())
    tailwind_means.append(cat_data['ssim_tailwind'].mean())

x = range(len(categories))
width = 0.35

plt.bar([i - width/2 for i in x], bootstrap_means, width, label='Bootstrap', alpha=0.8)
plt.bar([i + width/2 for i in x], tailwind_means, width, label='Tailwind', alpha=0.8)

plt.xlabel('Kategori Kompleksitas')
plt.ylabel('Rata-rata SSIM Score')
plt.title('Perbandingan SSIM: Bootstrap vs Tailwind')
plt.xticks(x, categories)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('ssim_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📈 Grafik perbandingan disimpan ke ssim_comparison.png")